In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output

In [2]:
app = Dash(__name__)

In [3]:
# Import data from excels into pandas
curprod = pd.ExcelFile(r'C:\Users\yelsk\Desktop\programming\plotly_Dash\sales_ex_DASHboard\currency product.xlsx')
cur = pd.read_excel(curprod, 'currency')
prod = pd.read_excel(curprod, 'products')

pur_or = pd.read_excel(r'C:\Users\yelsk\Desktop\programming\plotly_Dash\sales_ex_DASHboard\purchase orders.xlsx')
sa_p = pd.read_excel(r'C:\Users\yelsk\Desktop\programming\plotly_Dash\sales_ex_DASHboard\sales people.xlsx')

pur_or.head()

,Unnamed: 0,client group,sales person,date,product code,units sold,transaction currency
0,0,CL-5,J001,2020-10-03,CS002,8.0,ILS
1,1,CL-4,B004,2021-05-03,CS002,5.0,ILS
2,2,CL-9,J001,2018-08-07,CS002,24.0,ILS
3,3,CL-1,B001,2018-12-04,CS002,22.0,ILS
4,4,CL-11,J003,2017-07-19,WS001,29.0,ILS


In [4]:
#data exploration and cleaning
pur_or.columns

#pur_or['units sold'].unique()

#pur_or[pur_or['client group']=='-']
pur_or.dropna(subset=['sales person', 'product code', 'units sold'], inplace=True)

#cleaning out a non existing products' rows
pur_or = pur_or[pur_or['product code'] != 'Cc']
pur_or = pur_or[pur_or['product code'] != '--']

#dropping the additional *index column (original from excel) in all dataframes
cur.drop(['Unnamed: 0'], axis=1, inplace=True)
prod.drop(['Unnamed: 0'], axis=1, inplace=True)
pur_or.drop(['Unnamed: 0'], axis=1, inplace=True)
sa_p.drop(['Unnamed: 0'], axis=1, inplace=True)


In [5]:
pur_or['product code'].unique()

array(['CS002', 'WS001', 'SW001', 'SW002', 'CS001'], dtype=object)

In [6]:
#adding info from other tables to pur_or dataframe
pur_or['sales_person_name'] = pur_or['sales person'].map(sa_p.set_index('salesperson code')['name'])
pur_or['company branch'] = pur_or['sales person'].map(sa_p.set_index('salesperson code')['company branch'])

pur_or['product_unit_price'] = pur_or['product code'].map(prod.set_index('product code')['unit price, ILS'])
pur_or['product_manif_cost'] = pur_or['product code'].map(prod.set_index('product code')['manufacturing cost, ILS'])


In [7]:
#price paid per transaction
pur_or['transaction pay'] = pur_or['product_unit_price']*pur_or['units sold']
pur_or['transaction profit'] = (pur_or['product_unit_price']-pur_or['product_manif_cost'])*pur_or['units sold']


In [8]:
#adding lon and lat of a branch
for row, t in pur_or.iterrows():
    if pur_or.loc[row, 'company branch'] == 'Jerusalem':
        pur_or.loc[row,'latitude'] = 31.77
        pur_or.loc[row,'longitude'] = 35.21
    elif pur_or.loc[row,'company branch'] == 'Beer Sheva':
        pur_or.loc[row,'latitude'] = 31.25
        pur_or.loc[row,'longitude'] = 34.79
    else:
        pur_or.loc[row,'latitude'] = 32.79
        pur_or.loc[row,'longitude'] = 34.99

In [9]:
df = pur_or.groupby(['sales person', 'product code', 'company branch', 'latitude', 'longitude'])[['transaction pay', 'transaction profit']].sum()
df.reset_index(inplace=True) #otherwise it creates multiindex dataframe which plotly does not support

In [10]:
df.head(3)

,sales person,product code,company branch,latitude,longitude,transaction pay,transaction profit
0,B001,CS001,Beer Sheva,31.25,34.79,373750.0,153985.00
1,B001,CS002,Beer Sheva,31.25,34.79,298870.0,140343.06
2,B001,SW001,Beer Sheva,31.25,34.79,298068.0,40416.00


## Dash visualizations

In [11]:
# App layout - what (the items) user will see on the page
app.layout = html.Div([

    html.H1("Sales analysis Dashboard", style={'text-align': 'center'}),
    
    html.Div(id='output_text', children=[]),
    dcc.Graph(id='map', figure={}),
    html.Br(),
    
    dcc.Graph(id='salesp_bar_chart'),# figure={}),

    dcc.Dropdown(id="select_product_code",
                 
                 options=[
                     {"label": "CS001", "value": 'CS001'},
                     {"label": "CS002", "value": 'CS002'},
                     {"label": "SW001", "value": 'SW001'},
                     {"label": "SW002", "value": 'SW002'},
                     {"label": "WS001", "value": 'WS001'}],
                 multi=False,
                 value='CS001',
                 style={'width': "40%"}
                 ),



    dcc.Graph(id='indiv_prod_bar_chart', figure={})

])

In [12]:
# MAP callback
@app.callback(
    [Output(component_id='map', component_property='figure')], #goes to app.layout with the same id as the component_id of this line
    [Input(component_id='select_product_code', component_property='value')]
)
def sales_map(option_slctd): 
    #the argument/s fo the function is/are always should be the component property of the callback's input/s -- option_slctd in this case
    #the function should return always the component property of the output/s
    #number of arguments the function exepts is equal to number of inputs in @app.callback
    #number of objects the function returns is equal to number of outputs in @app.callback

    dff = df.copy()
    
    fig = px.density_mapbox(
        data_frame=dff,
        lat='latitude',
        lon='longitude',
        z = 'transaction profit',
        radius = 10,#'transaction profit',
        center = dict(lat=32, lon=34), zoom = 6,
        hover_name = 'company branch',
        mapbox_style = 'open-street-map',
        labels = 'transaction profit',
        color_continuous_scale = 'Rainbow',
        title = 'Sales Profit by Company Branch'
    )

    return [fig]

In [13]:
# Connect the Plotly graphs with Dash Components
@app.callback(
    [Output(component_id='salesp_bar_chart', component_property='figure')], #goes to app.layout with the same id as the component_id of this line
    [Input(component_id='select_product_code', component_property='value')]
)
def sales_people_bar(noinput): 

    dff = df.copy()
    
    #plotly bar
    fig = px.bar(df, x='sales person', y='transaction profit', color='product code')

    return [fig]

In [14]:
@app.callback(
    [#Output(component_id='output_container', component_property='children'), #goes to app.layout with the same id as the component_id of this line
     Output(component_id='indiv_prod_bar_chart', component_property='figure')], #goes to app.layout with the same id as the component_id of this line
    [Input(component_id='select_product_code', component_property='value')]
)
def sales_product_bar(option_slctd): 

    dff = df.copy()
    dff = dff[dff["product code"] == option_slctd]
    
    #plotly bar
    fig = px.bar(dff, x='sales person', y='transaction profit')

    return [fig]

In [15]:
if __name__ == '__main__':
    app.run_server(debug=True, port=8051) #to define the port so it doesn't clash with sone other app